In [1]:
import plotly.figure_factory as ff
import pandas as pd
import pickle
import plotly.graph_objs as go
import latextable
from texttable import Texttable
from strategy.strategy import (
    BuyAndHoldStrategy,
    MACDStrategy,
    RSIStrategy,
    ModelQuantilePredictionsStrategy,
    ModelGmadlPredictionsStrategy,
    ConcatenatedStrategies
)
from strategy.util import (
    get_data_windows,
    get_sweep_window_predictions,
    get_predictions_dataframe
)
from strategy.evaluation import (
    parameter_sweep,
    evaluate_strategy
)
from strategy.plotting import (
    plot_sweep_results
)

PADDING=5000
VALID_PART=0.2

In [2]:
data_windows_5min = get_data_windows(
    'wne-masters-thesis-testing',
    'btc-usdt-5m:latest',
    min_window=0, 
    max_window=5
)

data_windows_15min = get_data_windows(
    'wne-masters-thesis-testing',
    'btc-usdt-15m:latest',
    min_window=0, 
    max_window=5
)

data_windows_30min = get_data_windows(
    'wne-masters-thesis-testing',
    'btc-usdt-30m:latest',
    min_window=0, 
    max_window=5
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Downloading large artifact btc-usdt-5m:latest, 745.12MB. 12 files... 
wandb:   12 of 12 files downloaded.  
Done. 0:0:1.1
wandb: Downloading large artifact btc-usdt-15m:latest, 248.65MB. 12 files... 
wandb:   12 of 12 files downloaded.  
Done. 0:0:0.6
wandb: Downloading large artifact btc-usdt-30m:latest, 124.19MB. 12 files... 
wandb:   12 of 12 files downloaded.  
Done. 0:0:0.4


In [3]:
with open('cache/5min-best-strategies.pkl', 'rb') as inpt:
    best_strategies_5min = pickle.load(inpt)

with open('cache/15min-best-strategies.pkl', 'rb') as inpt:
    best_strategies_15min = pickle.load(inpt)

with open('cache/30min-best-strategies.pkl', 'rb') as inpt:
    best_strategies_30min = pickle.load(inpt)

## Best 5 strategies evaluation

In [5]:
def results_plot(buy_and_hold_concat, gmadl_5min_model_concat, rsi_5_min_concat, gmadl_15min_model_concat, rsi_30min_concat, macd_30min_concat, width=850, height=500, notitle=False):

    fig = go.Figure([
        go.Scatter(y=buy_and_hold_concat['portfolio_value'], x=buy_and_hold_concat['time'], name="Buy and Hold"),
        go.Scatter(y=gmadl_5min_model_concat['portfolio_value'], x=gmadl_5min_model_concat['time'], name="GMADL Informer Strategy (5min)"),
        go.Scatter(y=rsi_5_min_concat['portfolio_value'], x=rsi_5_min_concat['time'], name="RSI Strategy (5min)"),
        go.Scatter(y=gmadl_15min_model_concat['portfolio_value'], x=gmadl_15min_model_concat['time'], name='GMADL Informer Strategy (15min)'),
        go.Scatter(y=rsi_30min_concat['portfolio_value'], x=rsi_30min_concat['time'], name='RSI Strategy (30min)'),
        go.Scatter(y=macd_30min_concat['portfolio_value'], x=macd_30min_concat['time'], name='MACD Strategy (30min)')
        ])
    fig.update_layout(
        # title={
        # 'text': f"W{idx}-{INTERVAL}",
        # 'y':0.97,
        # 'x':0.5,
        # 'xanchor': 'center',
        # 'yanchor': 'top'} if not notitle else None,
        yaxis_title="Portfolio Value",
        xaxis_title="Date",
            font=dict(
            # family="Courier New, monospace",
            size=14,
        ),
        autosize=False,
        width=width,
        height=height,
        margin=dict(l=20, r=20, t=20, b=20),
        plot_bgcolor='white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="left",
            x=0.02
        )
    )
    fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
    )
    fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
    )
    # fig.write_image(f"images/eval-w{idx}-{INTERVAL}.png")
    fig.show()
    
def results_table(buy_and_hold_concat, gmadl_5min_model_concat, rsi_5_min_concat, gmadl_15min_model_concat, rsi_30min_concat, macd_30min_concat):
    table_eval_windows = Texttable()
    table_eval_windows.set_deco(Texttable.HEADER)
    table_eval_windows.set_cols_align(["l", "c","c", "c", "c", "c", "c", "c", "c", "c"])
    table_eval_windows.set_precision(3)

    table_eval_windows.header([
        "\\textbf{Strategy}",
        "\\textbf{VAL}",
        "\\textbf{ARC}",
        "\\textbf{ASD}",
        "\\textbf{IR*}",
        "\\textbf{MD}",
        "\\textbf{IR**}",
        "\\textbf{N}",
        "\\textbf{LONG}",
        "\\textbf{SHORT}",
    ])

    strategy_name_result = [
        ('Buy and Hold', buy_and_hold_concat),
        ('GMADL Informer (5min)', gmadl_5min_model_concat),
        ('RSI Strategy (5min)', rsi_5_min_concat),
        ('GMADL Informer (15min)', gmadl_15min_model_concat),
        ('RSI Strategy (30min)', rsi_30min_concat),
        ('MACD Strategy (30min)', macd_30min_concat)
    ]
    for strategy_name, result in strategy_name_result:
        table_eval_windows.add_row([
            strategy_name,
            result['value'],
            result['arc'],
            result['asd'],
            result['ir'],
            result['md'],
            result['mod_ir'],
            result['n_trades'],
            f"{result['long_pos']*100:.2f}\%",
            f"{result['short_pos']*100:.2f}\%",
        ])
    print(latextable.draw_latex(table_eval_windows))

In [7]:
# test_data_5min = pd.concat([data_windows_5min[0][0][-PADDING:]] + [data_window[1] for data_window in data_windows_5min])
# test_data_15min = pd.concat([data_windows_15min[0][0][-PADDING:]] + [data_window[1] for data_window in data_windows_15min])
# test_data_30min = pd.concat([data_windows_30min[0][0][-PADDING:]] + [data_window[1] for data_window in data_windows_30min])

# buy_and_hold_concat = evaluate_strategy(test_data_5min, BuyAndHoldStrategy(), padding=PADDING, interval='5min')
# gmadl_5min_model_concat = evaluate_strategy(test_data_5min, ConcatenatedStrategies(len(data_windows_5min[0][1]), [s[0] for s in best_strategies_5min['gmadl_model']], padding=PADDING), padding=PADDING, interval='5min')
# gmadl_15min_model_concat = evaluate_strategy(test_data_15min, ConcatenatedStrategies(len(data_windows_15min[0][1]), [s[0] for s in best_strategies_15min['gmadl_model']], padding=PADDING), padding=PADDING, interval='15min')
# rsi_5_min_concat = evaluate_strategy(test_data_5min, ConcatenatedStrategies(len(data_windows_5min[0][1]), [s[0] for s in best_strategies_5min['rsi_strategies']], padding=PADDING), padding=PADDING, interval='5min')
# rsi_30min_concat = evaluate_strategy(test_data_30min, ConcatenatedStrategies(len(data_windows_30min[0][1]), [s[0] for s in best_strategies_30min['rsi_strategies']], padding=PADDING), padding=PADDING, interval='30min')
# macd_30min_concat = evaluate_strategy(test_data_30min, ConcatenatedStrategies(len(data_windows_30min[0][1]), [s[0] for s in best_strategies_30min['macd_strategies']], padding=PADDING), padding=PADDING, interval='30min')

# results_table(buy_and_hold_concat, gmadl_5min_model_concat, rsi_5_min_concat, gmadl_15min_model_concat, rsi_30min_concat, macd_30min_concat)
# results_plot(buy_and_hold_concat, gmadl_5min_model_concat, gmadl_15min_model_concat, rsi_5_min_concat, rsi_30min_concat, macd_30min_concat, width=1200, notitle=True)

## Statistical significance

In [10]:
test_data_5min = pd.concat([data_windows_5min[0][0][-PADDING:]] + [data_window[1] for data_window in data_windows_5min])
test_data_15min = pd.concat([data_windows_15min[0][0][-PADDING:]] + [data_window[1] for data_window in data_windows_15min])
test_data_30min = pd.concat([data_windows_30min[0][0][-PADDING:]] + [data_window[1] for data_window in data_windows_30min])

buy_and_hold_5min = evaluate_strategy(test_data_5min, BuyAndHoldStrategy(), padding=PADDING, interval='5min')
buy_and_hold_15min = evaluate_strategy(test_data_15min, BuyAndHoldStrategy(), padding=PADDING, interval='15min')
buy_and_hold_30min = evaluate_strategy(test_data_30min, BuyAndHoldStrategy(), padding=PADDING, interval='30min')
gmadl_5min_model_concat = evaluate_strategy(test_data_5min, ConcatenatedStrategies(len(data_windows_5min[0][1]), [s[0] for s in best_strategies_5min['gmadl_model']], padding=PADDING), padding=PADDING, interval='5min')
gmadl_15min_model_concat = evaluate_strategy(test_data_15min, ConcatenatedStrategies(len(data_windows_15min[0][1]), [s[0] for s in best_strategies_15min['gmadl_model']], padding=PADDING), padding=PADDING, interval='15min')
rsi_5_min_concat = evaluate_strategy(test_data_5min, ConcatenatedStrategies(len(data_windows_5min[0][1]), [s[0] for s in best_strategies_5min['rsi_strategies']], padding=PADDING), padding=PADDING, interval='5min')
rsi_30min_concat = evaluate_strategy(test_data_30min, ConcatenatedStrategies(len(data_windows_30min[0][1]), [s[0] for s in best_strategies_30min['rsi_strategies']], padding=PADDING), padding=PADDING, interval='30min')
macd_30min_concat = evaluate_strategy(test_data_30min, ConcatenatedStrategies(len(data_windows_30min[0][1]), [s[0] for s in best_strategies_30min['macd_strategies']], padding=PADDING), padding=PADDING, interval='30min')

In [55]:
import numpy as np
import scipy.stats

def ttest(benchmark, strategy):
    sigma = np.std(strategy['portfolio_value'] - benchmark['portfolio_value'])
    N = len(strategy['strategy_returns'])
    tt = (strategy['ir'] - benchmark['ir']) / (sigma / np.sqrt(N))

    pval = scipy.stats.t.sf(np.abs(tt), N-1)*2

    return tt, pval, sigma, N

gmadl_5min_ttest = ttest(buy_and_hold_5min, gmadl_5min_model_concat)
gmadl_15min_ttest = ttest(buy_and_hold_15min, gmadl_15min_model_concat)
rsi_5min_ttest = ttest(buy_and_hold_5min, rsi_5_min_concat)
rsi_30min_ttest = ttest(buy_and_hold_30min, rsi_30min_concat)
macd_30min_ttest = ttest(buy_and_hold_30min, macd_30min_concat)

for name, result in [
    ("GMADL (5min)", gmadl_5min_ttest),
    ("GMADL (15 min)", gmadl_15min_ttest),
    ("RSI (5 min)", rsi_5min_ttest),
    ("RSI (30 min)", rsi_30min_ttest),
    ("MACD (30 min)", macd_30min_ttest)
]:
    print(f"{name} & {result[3]} & {result[2]:.6f} & {result[0]:.2f} & {result[1]:.6f}*** \\\\")

GMADL (5min) & 311040 & 2.820834 & 375.84 & 0.000000*** \\
GMADL (15 min) & 103680 & 0.558743 & 408.13 & 0.000000*** \\
RSI (5 min) & 311040 & 0.648741 & 662.77 & 0.000000*** \\
RSI (30 min) & 51840 & 1.065079 & 258.49 & 0.000000*** \\
MACD (30 min) & 51840 & 0.326504 & 174.34 & 0.000000*** \\


## TOP N Strategies Sensistivity Analisys

In [56]:
# def results_for_strats(data_windows, best_strategies, interval='5min', top_n=10):
#     test_data = pd.concat([data_windows[0][0][-PADDING:]] + [data_window[1] for data_window in data_windows])

    
#     buy_and_hold_concat = evaluate_strategy(test_data, BuyAndHoldStrategy(), padding=PADDING, interval=interval)
#     macd_concat = [evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[x] for s in best_strategies['macd_strategies']], padding=PADDING), padding=PADDING, interval=interval) for x in range(top_n)]
#     rsi_concat = [evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[x] for s in best_strategies['rsi_strategies']], padding=PADDING), padding=PADDING, interval=interval) for x in range(top_n)]
#     quantile_model_concat = [evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[x] for s in best_strategies['quantile_model']], padding=PADDING), padding=PADDING, interval=interval) for x in range(top_n)]
#     gmadl_model_concat = [evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[x] for s in best_strategies['gmadl_model']], padding=PADDING), padding=PADDING, interval=interval) for x in range(top_n)]

#     z = list(reversed([
#         list(reversed([round(buy_and_hold_concat['mod_ir'], 3)]*top_n)),
#         list(reversed([round(x['mod_ir'], 3) for x in macd_concat])),
#         list(reversed([round(x['mod_ir'], 3) for x in rsi_concat])),
#         list(reversed([round(x['mod_ir'], 3) for x in quantile_model_concat])),
#         list(reversed([round(x['mod_ir'], 3) for x in gmadl_model_concat])),
#     ]))
#     x = list(reversed(range(1, top_n+1)))
#     y = list(reversed([
#         "Buy and Hold",
#         "MACD Strategy",
#         "RSI Strategy",
#         "Quantile Informer",
#         "Gmadl Informer"
#     ]))
#     # 'Portland'
#     fig = ff.create_annotated_heatmap(z, x=x, y=y, colorscale='thermal', zmid=buy_and_hold_concat['mod_ir'])
#     fig.update_layout(
#         margin=dict(l=20, r=20, b=20, t=20),
#         width=1100,
#         height=450,
#         font=dict(
#         #    family="Courier New, monospace",
#             size=18,  # Set the font size here
#             # color="RebeccaPurple"
#         )
#     )
#     fig.show()

# results_for_strats(data_windows_5min, best_strategies_5min, interval='5min', top_n=10) 
# results_for_strats(data_windows_15min, best_strategies_15min, interval='15min', top_n=10) 
# results_for_strats(data_windows_30min, best_strategies_30min, interval='30min', top_n=10) 


## Validation size for: GMADL Informer (5min)

In [27]:
with open('cache/5min-best-strategies-01.pkl', 'rb') as inpt:
    best_strategies_5min_01 = pickle.load(inpt)

with open('cache/5min-best-strategies.pkl', 'rb') as inpt:
    best_strategies_5min_02 = pickle.load(inpt)

with open('cache/5min-best-strategies-03.pkl', 'rb') as inpt:
    best_strategies_5min_03 = pickle.load(inpt)

with open('cache/5min-best-strategies-04.pkl', 'rb') as inpt:
    best_strategies_5min_04 = pickle.load(inpt)

In [5]:
# print(best_strategies_5min_02['gmadl_model'][0][0].info())
# print(best_strategies_5min_03['gmadl_model'][0][0].info())

{'strategy_name': 'Unknown model', 'future': 1, 'target': 'close_price', 'enter_long': 0.004, 'exit_long': None, 'enter_short': -0.005, 'exit_short': None}
{'strategy_name': 'Unknown model', 'future': 1, 'target': 'close_price', 'enter_long': 0.007, 'exit_long': -0.003, 'enter_short': None, 'exit_short': None}


In [48]:
def results_plot2(result_buyandhold, results, names, width=850, height=500):

    fig = go.Figure([
        go.Scatter(y=result_buyandhold['portfolio_value'], x=result_buyandhold['time'], name="Buy and Hold")] +[
        go.Scatter(y=result['portfolio_value'], x=result['time'], name=name) for name, result in zip(names, results)
    ])
        # go.Scatter(y=result_macd['portfolio_value'], x=result_macd['time'], name="MACD Strategy"),
        # go.Scatter(y=result_rsi['portfolio_value'], x=result_rsi['time'], name="RSI Strategy"),
        # go.Scatter(y=result_quantile_model['portfolio_value'], x=result_quantile_model['time'], name='Quantile Informer Strategy'),
        # go.Scatter(y=result_gmadl_model['portfolio_value'], x=result_gmadl_model['time'], name='GMADL Informer Strategy')
        # ])
    fig.update_layout(
        yaxis_title="Portfolio Value",
        xaxis_title="Date",
            font=dict(
            # family="Courier New, monospace",
            size=14,
        ),
        autosize=False,
        width=width,
        height=height,
        margin=dict(l=20, r=20, t=20, b=20),
        plot_bgcolor='white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="left",
            x=0.02
        )
    )
    fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
    )
    fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
    )
    fig.show()
    
def results_table2(result_buyandhold, results, names):
    table_eval_windows = Texttable()
    table_eval_windows.set_deco(Texttable.HEADER)
    table_eval_windows.set_cols_align(["l", "c","c", "c", "c", "c", "c", "c", "c", "c"])
    table_eval_windows.set_precision(2)

    table_eval_windows.header([
        "\\textbf{Strategy}",
        "\\textbf{VAL}",
        "\\textbf{ARC}",
        "\\textbf{ASD}",
        "\\textbf{IR*}",
        "\\textbf{MD}",
        "\\textbf{IR**}",
        "\\textbf{N}",
        "\\textbf{LONG}",
        "\\textbf{SHORT}",
    ])

    strategy_name_result = [
        ('Buy and Hold', result_buyandhold)] + list(zip(names, results))
    for strategy_name, result in strategy_name_result:
        table_eval_windows.add_row([
            strategy_name,
            result['value'],
            f"{result['arc']*100:.1f}\%",
            f"{result['asd']*100:.1f}\%",
            result['ir'],
            f"{result['md']*100:.1f}\%",
            result['mod_ir'],
            result['n_trades'],
            f"{result['long_pos']*100:.1f}\%",
            f"{result['short_pos']*100:.1f}\%",
        ])
    print(latextable.draw_latex(table_eval_windows))

In [37]:
def results_for_strats(data_windows, best_strategies, names, interval='5min', strat_name='gmadl_model'):
    test_data = pd.concat([data_windows[0][0][-PADDING:]] + [data_window[1] for data_window in data_windows])

    buy_and_hold_concat = evaluate_strategy(test_data, BuyAndHoldStrategy(), padding=PADDING, interval=interval)
    
    evaluation_results = []
    for best_strat in best_strategies:
        evaluation_results.append(
            evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[0] for s in best_strat[strat_name]], padding=PADDING), padding=PADDING, interval=interval)
        )
    results_plot(buy_and_hold_concat, evaluation_results, names, width=1200)
    results_table2(buy_and_hold_concat, evaluation_results, names)


# results_for_strats(data_windows_5min, [
#     best_strategies_5min_01,
#     best_strategies_5min_02,
#     best_strategies_5min_03,
#     best_strategies_5min_04
#     ], [
#         "3 months",
#         "6 months",
#         "9 months",
#         "12 months",
#     ])


In [40]:
with open('cache/30min-best-strategies-01.pkl', 'rb') as inpt:
    best_strategies_30min_01 = pickle.load(inpt)

with open('cache/30min-best-strategies.pkl', 'rb') as inpt:
    best_strategies_30min_02 = pickle.load(inpt)

with open('cache/30min-best-strategies-03.pkl', 'rb') as inpt:
    best_strategies_30min_03 = pickle.load(inpt)

with open('cache/30min-best-strategies-04.pkl', 'rb') as inpt:
    best_strategies_30min_04 = pickle.load(inpt)

In [3]:
# results_for_strats(data_windows_30min, [
#     best_strategies_30min_01,
#     best_strategies_30min_02,
#     best_strategies_30min_03,
#     best_strategies_30min_04
#     ], [
#         "3 months",
#         "6 months",
#         "9 months",
#         "12 months",
#     ], strat_name='rsi_strategies', interval='30min')

In [4]:
# test_data = pd.concat([data_window[0] for data_window in data_windows_5min[:1]])
# buy_and_hold_concat = evaluate_strategy(test_data, BuyAndHoldStrategy(), padding=PADDING)

# res = evaluate_strategy(test_data, ConcatenatedStrategies(1, [s[0] for s in best_strategies_5min_02['gmadl_model'][:1]], padding=PADDING), padding=PADDING, interval='5min')
# results_plot(buy_and_hold_concat, [res], width=1200)

### Different number of test windows

In [36]:
with open('cache/5min-best-strategies-short.pkl', 'rb') as inpt:
    best_strategies_5min_short = pickle.load(inpt)

with open('cache/5min-best-strategies-long.pkl', 'rb') as inpt:
    best_strategies_5min_long = pickle.load(inpt)

with open('cache/30min-best-strategies-short.pkl', 'rb') as inpt:
    best_strategies_30min_short = pickle.load(inpt)

with open('cache/30min-best-strategies-long.pkl', 'rb') as inpt:
    best_strategies_30min_long = pickle.load(inpt)

In [39]:
data_windows_5min_short = get_data_windows(
    'wne-masters-thesis-testing',
    'btc-usdt-5m-short:latest',
    min_window=0, 
    max_window=11
)

data_windows_5min_long = get_data_windows(
    'wne-masters-thesis-testing',
    'btc-usdt-5m-long:latest',
    min_window=0, 
    max_window=2
)

data_windows_30min_short = get_data_windows(
    'wne-masters-thesis-testing',
    'btc-usdt-30m-short:latest',
    min_window=0, 
    max_window=11
)

data_windows_30min_long = get_data_windows(
    'wne-masters-thesis-testing',
    'btc-usdt-30m-long:latest',
    min_window=0, 
    max_window=2
)

wandb: Downloading large artifact btc-usdt-5m-short:latest, 1341.36MB. 24 files... 
wandb:   24 of 24 files downloaded.  
Done. 0:0:0.7
wandb: Downloading large artifact btc-usdt-5m-long:latest, 446.99MB. 6 files... 
wandb:   6 of 6 files downloaded.  
Done. 0:0:1.1
wandb: Downloading large artifact btc-usdt-30m-short:latest, 223.56MB. 24 files... 
wandb:   24 of 24 files downloaded.  
Done. 0:0:0.7
wandb: Downloading large artifact btc-usdt-30m-long:latest, 74.51MB. 6 files... 
wandb:   6 of 6 files downloaded.  
Done. 0:0:0.4


In [57]:
# def results_for_strats_different_windows(buy_and_hold_data_windows, *args, interval='5min'):
#         evaluation_results = []
#         names = []
#         buy_and_hold_data = pd.concat([buy_and_hold_data_windows[0][0][-PADDING:]] + [data_window[1] for data_window in buy_and_hold_data_windows])
#         buy_and_hold_concat = evaluate_strategy(buy_and_hold_data, BuyAndHoldStrategy(), padding=PADDING, interval=interval)
#         for data_windows, strategy, name in args:
#             test_data = pd.concat([data_windows[0][0][-PADDING:]] + [data_window[1] for data_window in data_windows])
#             #  buy_and_hold_concat = evaluate_strategy(test_data, BuyAndHoldStrategy(), padding=PADDING, interval=interval)
#             evaluation_results.append(
#                  evaluate_strategy(test_data, ConcatenatedStrategies(len(data_windows[0][1]), [s[0] for s in strategy], padding=PADDING), padding=PADDING, interval=interval
#             ))
#             names.append(name)
        
#         results_plot2(buy_and_hold_concat, evaluation_results, names, width=1200)
#         results_table2(buy_and_hold_concat, evaluation_results, names)

# results_for_strats_different_windows(
#        data_windows_30min,
#       (data_windows_30min_short, best_strategies_30min_short['rsi_strategies'], '12 windows'),
#       (data_windows_30min, best_strategies_30min['rsi_strategies'], '6 windows'),
#       (data_windows_30min_long, best_strategies_30min_long['rsi_strategies'], '3 windows'),
#       interval='30min'
# )

# results_for_strats_different_windows(
#        data_windows_5min,
#       (data_windows_5min_short, best_strategies_5min_short['gmadl_model'], '12 windows'),
#       (data_windows_5min, best_strategies_5min['gmadl_model'], '6 windows'),
#       (data_windows_5min_long, best_strategies_5min_long['gmadl_model'], '3 windows'),
#       interval='5min'
# )